In [25]:
import pandas as pd
import os
import tensorflow as tf
from bokeh.plotting import figure, show
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import glob

In [5]:
datagen = ImageDataGenerator(
    rescale = 1./255
)

In [4]:
data_images = pd.DataFrame(os.listdir("../../datasets/brain_tumors/"), columns=["File_Names"])
data_images

,File_Names
0,glioma_tumor
1,meningioma_tumor
2,normal
3,pituitary_tumor


In [3]:
# train_generator = datagen.flow_from_directory(
#     "../../datasets/brain_tumors/",
#     target_size=(256, 256),
#     batch_size=32,
#     # validation_split=0.2,
#     class_mode="categorical",
#     subset="training"
# )
# validation_generator = datagen.flow_from_directory(
#     "../../datasets/brain_tumors/",
#     target_size=(256, 256),
#     batch_size=32,
#     # validation_split=0.2,
#     class_mode="categorical",
#     subset="validation"
# )

In [6]:
train = tf.keras.preprocessing.image_dataset_from_directory(
    "../../datasets/brain_tumors/",
    validation_split=0.2,
    seed=100,
    subset="training",
    image_size=(256, 256),
    batch_size=32
)
validation = tf.keras.preprocessing.image_dataset_from_directory(
    "../../datasets/brain_tumors/",
    validation_split=0.2,
    seed=100,
    subset="validation",
    image_size=(256, 256),
    batch_size=32
)

Found 3096 files belonging to 4 classes.
Using 2477 files for training.
Found 3096 files belonging to 4 classes.
Using 619 files for validation.


In [7]:
classes = train.class_names
print(classes)

['glioma_tumor', 'meningioma_tumor', 'normal', 'pituitary_tumor']


In [15]:
model = keras.Sequential([
    layers.Conv2D(64, (3, 3), activation="relu", input_shape=(256, 256, 3)),
    layers.MaxPooling2D(4, 4),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D(4, 4),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(4, activation='softmax')
])

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(train, epochs=5, validation_data=validation)
#OBS: Demorando demais e acuracia abaixo de 30%

Epoch 1/5


78/78 [==============================] - 125s 2s/step - loss: 1.0392 - accuracy: 0.5478 - val_loss: 0.9183 - val_accuracy: 0.6123
Epoch 2/5
78/78 [==============================] - 111s 1s/step - loss: 0.7885 - accuracy: 0.6750 - val_loss: 0.7691 - val_accuracy: 0.6834
Epoch 3/5
78/78 [==============================] - 112s 1s/step - loss: 0.5436 - accuracy: 0.7808 - val_loss: 0.6668 - val_accuracy: 0.7496
Epoch 4/5
78/78 [==============================] - 112s 1s/step - loss: 0.4148 - accuracy: 0.8417 - val_loss: 0.6655 - val_accuracy: 0.7690
Epoch 5/5
78/78 [==============================] - 112s 1s/step - loss: 0.3057 - accuracy: 0.8849 - val_loss: 0.7344 - val_accuracy: 0.7787


In [19]:
score, acc = model.evaluate(validation)
print(score)
print(acc)

20/20 [==============================] - 7s 346ms/step - loss: 0.7344 - accuracy: 0.7787
0.7343992590904236
0.7786752581596375


In [20]:
hist = pd.DataFrame(history.history)
hist

,loss,accuracy,val_loss,val_accuracy
0,1.039239,0.547840,0.918332,0.612278
1,0.788536,0.675010,0.769107,0.683360
2,0.543613,0.780783,0.666839,0.749596
3,0.414790,0.841744,0.665525,0.768982
4,0.305700,0.884941,0.734399,0.778675


20/20 [==============================] - 9s 438ms/step


array([[2.55664252e-02, 9.26200151e-01, 1.32110505e-03, 4.69122417e-02],
       [8.63975671e-04, 1.27570909e-02, 2.82284964e-05, 9.86350715e-01],
       [5.24219777e-03, 1.71700809e-02, 9.77565408e-01, 2.23134139e-05],
       ...,
       [2.59727496e-03, 5.97325899e-03, 9.91429389e-01, 1.14228484e-07],
       [1.61685869e-02, 1.85702574e-02, 9.09438431e-01, 5.58227636e-02],
       [3.91365821e-03, 9.96048391e-01, 2.38393623e-05, 1.41828014e-05]],
      dtype=float32)